In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel("data/patient1/fn_eeg.xlsx")

In [4]:
df.head()

,Millis,HardwareSeconds,ch_1,ch_2,ch_3,ch_4,ch_5,ch_6,ch_7,ch_8,...,ch_15,ch_16,ch_17,ch_18,ch_19,ch_20,ch_21,ch_22,ch_23,ch_24
0,51,0,186.9,130.2292,238.6686,-13.18469,326.7506,295.4188,233.2859,337.9861,...,351.5019,219.51960,-9.9,-5.7,481.6910,507.4703,-9.0,375.1644,103.1462,589.8848
1,51,0,188.4,130.8266,235.6815,-12.88504,326.7506,301.1173,234.7813,337.9861,...,353.8889,217.42030,-9.9,-5.1,499.3981,509.8711,-8.7,369.4710,110.6206,586.8936
2,52,0,181.2,129.0344,241.3569,-13.48434,327.0501,314.6136,239.5667,349.3521,...,360.1551,104.36180,-9.9,-5.1,447.4774,530.8782,-8.7,387.1505,116.3011,598.8588
3,52,0,168.0,128.7357,246.4350,-12.88504,328.5476,322.4114,239.2676,354.4368,...,368.5100,40.78506,-10.2,-5.4,409.0622,528.7774,-9.0,394.3422,120.1878,608.4309
4,52,0,167.4,129.0344,239.2660,-13.18469,324.9537,311.9143,238.9685,343.6691,...,366.4213,83.36946,-10.2,-5.4,418.3659,522.4753,-9.0,387.7498,113.9093,596.4657


In [5]:
eeg_raw = df.iloc[:,2:].values

In [6]:
eeg_raw.shape

(42851, 24)

In [7]:
from preprocess_omer import preprocess_eeg

eeg_100hz = preprocess_eeg(eeg_raw)

In [8]:
eeg_100hz

array([[ -2.18096968,  -2.88914887,   2.64423846, ...,  -0.7602516 ,
         -1.67391416,  -1.57915052],
       [ 49.16122004,  41.53156248,  87.35337167, ..., 140.78431683,
         43.98287203, 212.6161965 ],
       [ 43.43612748,  32.02391635,  66.61028332, ..., 120.00093317,
         38.80617192, 181.17157226],
       ...,
       [  1.03611149,   5.77680967,  25.99821003, ...,   5.56089726,
          8.46781301,  32.54852699],
       [ -4.13902275,   4.41987283,  28.01793352, ...,  15.77431421,
         13.07275367,  40.00483235],
       [  2.72458089,   3.84458571,   6.82734717, ...,   0.34502901,
          3.19503383,   6.47633399]], shape=(14284, 24))

In [9]:
timestamps = df["Millis"][::3]

In [11]:
timestamps.index = range(len(timestamps))

In [12]:
timestamps

0            51
1            52
2            52
3            52
4            52
          ...  
14279    142814
14280    142815
14281    142815
14282    142815
14283    142815
Name: Millis, Length: 14284, dtype: int64

In [13]:
dfcs = pd.read_excel("data/patient1/fn_cs.xlsx")

In [15]:
dfcs.head()

,Millis,Rating,ChangePerSec
0,85,0,-0.096045
1,85,0,0.242432
2,85,0,0.242432
3,85,0,0.242432
4,85,0,0.242432


In [23]:
from preprocess_omer import compute_cs_targets

compute_cs_targets(dfcs["ChangePerSec"])[0]

array([0.       , 0.2424316, 0.2424316, ..., 0.2424316, 0.2424316,
       0.2424316], shape=(16872,))

In [26]:
total = eeg_100hz.shape[0]

num_windows = total//300

eeg_100hz = eeg_100hz[:num_windows*300,:]
windowed_eeg = eeg_100hz.reshape(num_windows,300,-1)

In [27]:
windowed_eeg

array([[[-2.18096968e+00, -2.88914887e+00,  2.64423846e+00, ...,
         -7.60251596e-01, -1.67391416e+00, -1.57915052e+00],
        [ 4.91612200e+01,  4.15315625e+01,  8.73533717e+01, ...,
          1.40784317e+02,  4.39828720e+01,  2.12616196e+02],
        [ 4.34361275e+01,  3.20239163e+01,  6.66102833e+01, ...,
          1.20000933e+02,  3.88061719e+01,  1.81171572e+02],
        ...,
        [ 1.73047126e+00, -3.92905320e+00, -1.17448523e+00, ...,
         -8.02956142e-01, -5.02760735e+00, -9.10101532e-01],
        [ 8.43758311e-01, -6.52856706e+00, -3.67271604e+00, ...,
         -1.81126565e+00, -5.74382246e+00, -5.26987940e+00],
        [ 3.57761871e+00, -1.99145899e+00, -1.55170512e+00, ...,
         -3.09652360e+00, -1.25140399e+00,  1.16601485e+00]],

       [[ 7.40890551e+00,  6.44184249e-01,  1.21400373e+00, ...,
          3.14462416e+00, -1.32272975e+00,  4.63756776e+00],
        [ 6.41008130e+00, -4.88371313e+00, -1.72222037e+00, ...,
         -3.09182580e+00, -5.45326719e

In [43]:
from preprocess_omer import compute_psd_batch

windowed_eeg = windowed_eeg.transpose(0,2,1)
psd = compute_psd_batch(windowed_eeg)

/home/sinan/Desktop/pyt/test/EEG_Cybersickness_Estimation_VR-Beyond_Subjectivity_Reproduction/preprocess_omer.py:258: RuntimeWarning: Not adaptively combining the spectral estimators due to a low number of tapers (2 < 3).
  psd, f = psd_array_multitaper(
/home/sinan/Desktop/pyt/test/EEG_Cybersickness_Estimation_VR-Beyond_Subjectivity_Reproduction/preprocess_omer.py:258: RuntimeWarning: Not adaptively combining the spectral estimators due to a low number of tapers (2 < 3).
  psd, f = psd_array_multitaper(
/home/sinan/Desktop/pyt/test/EEG_Cybersickness_Estimation_VR-Beyond_Subjectivity_Reproduction/preprocess_omer.py:258: RuntimeWarning: Not adaptively combining the spectral estimators due to a low number of tapers (2 < 3).
  psd, f = psd_array_multitaper(
/home/sinan/Desktop/pyt/test/EEG_Cybersickness_Estimation_VR-Beyond_Subjectivity_Reproduction/preprocess_omer.py:258: RuntimeWarning: Not adaptively combining the spectral estimators due to a low number of tapers (2 < 3).
  psd, f = ps

In [44]:
psd = psd[0]

In [45]:
psd

array([[[2.64951477e+01, 2.66844463e+01, 2.68737431e+01, ...,
         4.52211164e-02, 5.50352074e-02, 6.48492947e-02],
        [2.71186485e+01, 2.90721550e+01, 3.10256596e+01, ...,
         8.15686360e-02, 9.07325149e-02, 9.98963937e-02],
        [7.01566124e+00, 9.26986504e+00, 1.15240698e+01, ...,
         1.04290783e-01, 1.08969465e-01, 1.13648146e-01],
        ...,
        [9.47815628e+01, 1.10946030e+02, 1.27110497e+02, ...,
         2.22709745e-01, 2.78148174e-01, 3.33586603e-01],
        [6.19123955e+01, 6.21403465e+01, 6.23682938e+01, ...,
         1.24351643e-02, 1.29442746e-02, 1.34533849e-02],
        [1.86383224e+02, 2.07947662e+02, 2.29512100e+02, ...,
         4.31510717e-01, 3.48204494e-01, 2.64898241e-01]],

       [[9.16980326e-01, 2.36468577e+00, 3.81239128e+00, ...,
         2.84413714e-03, 2.94235861e-03, 3.04058008e-03],
        [2.31041044e-01, 8.00874054e-01, 1.37070704e+00, ...,
         7.10123731e-03, 6.58948021e-03, 6.07772358e-03],
        [1.59398735e+00, 